In [ ]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(trainx,trainy),(testx,testy)=mnist.load_data()
plt.imshow(trainx[4], cmap=plt.get_cmap('gray'))
plt.show()
num_classes = 10

In [ ]:
trainx = trainx.reshape(60000, 28, 28, 1)
testx  = testx.reshape(10000, 28, 28, 1)
trainx = trainx/255.0
testx = testx / 255.0
trainy = to_categorical(trainy)
testy = to_categorical(testy)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(28, 28, 1), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
batch_size = 32
epochs = 5
model.fit(trainx, trainy,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(testx, testy))

In [ ]:
val_loss, val_acc = model.evaluate(testx, testy)
print(val_loss, val_acc)

In [ ]:
model.save('my_cnn_model.h5')

In [ ]:
path = "E:\Sudoku_Solver\sudoku_img2.jpg"
img = cv2.imread(path)
try:
    img = cv2.resize(img,(28*9,28*9))
except:
    pass
imgBlank = np.zeros((28*9,28*9,3), np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(gray,(1,1),cv2.BORDER_DEFAULT)
thresh = cv2.adaptiveThreshold(img_blur,255,1,1,11,2)
#cv2.imshow("thresh", thresh)
#cv2.waitKey(0)

In [ ]:
img_contours = img.copy()
img_big_contour = img.copy()
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img_contours, contours, -1, (0,255,8), 1)
#cv2.imshow("img_contours", img_contours)
#cv2.waitKey(0)

In [ ]:
def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area>max_area and len(approx)==4:
                biggest = approx
                max_area = area
    return biggest, max_area

In [ ]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2), dtype=np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

In [ ]:
biggest, maxArea = biggest_contour(contours)
if biggest.size != 0:
    biggest = reorder(biggest)
    cv2.drawContours(img_big_contour, biggest, -1, (8,255,0), 5)
    pts1 = np.float32(biggest) # prepare points for Warp
    pts2 = np.float32([[0,0], [9*28,0], [0,9*28], [9*28,9*28]]) # prepare points for Warp
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpedColored = cv2.warpPerspective(img, matrix, (9*28,9*28))
    imgDetectedDigits = imgBlank.copy()
    imgWarpedColored = cv2.cvtColor(imgWarpedColored, cv2.COLOR_BGR2GRAY)
    imgWarpedColored = cv2.adaptiveThreshold(imgWarpedColored, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 21, 4)

In [ ]:
cv2.imshow("img_big_contour", imgWarpedColored)
cv2.waitKey(0)

In [ ]:
def splitBoxes(img):
    rows = np.vsplit(img,9)
    boxes = []
    for r in rows:
        col = np.hsplit(r, 9)
        for box in col:
            boxes.append(box)
    return boxes

In [ ]:
imgSolvedDigits = imgBlank.copy()
boxes = splitBoxes(imgWarpedColored)
print(len(boxes))
print(type(boxes[0]))
print(boxes[0].shape)

In [ ]:
i=0
while i < 81:
    boxes[i] = cv2.resize(boxes[i][5:25,4:25],(28,28))
    
    i+=1
count = 0
for box in boxes:
    cv2.imwrite(f"E:\\Sudoku_Solver\\SudokuCells\\cell{count}.jpg",box)
    count += 1

In [ ]:
sudoku_model = load_model('my_cnn_model.h5')

In [ ]:
def predictnums(img):
    predicted = sudoku_model.predict_classes(img)
    return predicted[0]

In [ ]:
def avg_brightness(img_array):
    rows = 28
    cols = 28
    sum = 0
    for i in range(rows):
        for j in range(cols):
            sum += img_array[i,j]
    avg = sum/(rows*cols)
    return avg

In [ ]:
def sudoku_dig():
    rows,columns = 9,9
    fig = plt.figure(figsize=(9, 9))
    for i in range(0,81):
        img_array = cv2.imread(f"./SudokuCells/cell{i}.jpg", cv2.IMREAD_GRAYSCALE)
        img_array = cv2.bitwise_not(img_array)
        img_array = cv2.resize(img_array,(28,28))
        img_array = cv2.GaussianBlur(img_array,(1,1),cv2.BORDER_DEFAULT)
        proc_img = img_array/255
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(proc_img, cmap='gray')
        plt.axis('off')
    plt.show()
    
    cell_digits,num = [],0
    i=0
    
    #for box in boxes:
    for i in range(0,81):
        img_array = cv2.imread(f"./SudokuCells/cell{i}.jpg", cv2.IMREAD_GRAYSCALE)
        img_array = cv2.bitwise_not(img_array)
        img_array = cv2.resize(img_array,(28,28))
        img_array = cv2.GaussianBlur(img_array,(1,1),cv2.BORDER_DEFAULT)
        backtorgb = cv2.cvtColor(img_array.copy(),cv2.COLOR_GRAY2RGB)
        hsv = cv2.cvtColor(backtorgb, cv2.COLOR_BGR2HSV)
        brightness = avg_brightness(hsv[:,:,2])
        print(f"Brightness{i} : ", brightness)
        proc_img = img_array/255
        proc_img = proc_img.reshape(-1,28,28,1)
        if brightness < 12:
            num = 0
        else:
            num = predictnums(proc_img)
        cell_digits.append(num)
    n = 9
    cell_digits = [cell_digits[i:i+n] for i in range(0, len(cell_digits), n)] 
    return cell_digits

In [ ]:
def get_board(board_img):
    for i in range(len(board_img)):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - - - ")
        for j in range(len(board_img[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")
            if j == 8:
                print(board_img[i][j])
            else:
                print(str(board_img[i][j]) + " ", end="")

In [ ]:
puzzle = sudoku_dig()

In [ ]:
get_board(puzzle)